# Installing required files

In [1]:
%pip install sacremoses==0.0.53
%pip install datasets
%pip install transformers
%pip install torch torchvision torchaudio
%pip install datasets
%pip install nltk
%pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=e229c8c25a41e64e2cd23c1c651e95e2fb500e2a0ee8000a495207b9339d3149
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Importing required files

In [2]:
import nltk
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import pipeline
from rouge_score import rouge_scorer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.translate.meteor_score import meteor_score
from google.colab import drive
from datasets import load_metric
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Mount Google Drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


# Define the path to the CSV file

In [19]:
path_to_file = '/content/drive/MyDrive/train501entries.csv'

# Load dataset from CSV

In [20]:
xsum_dataset = load_dataset(
    'csv',
    data_files=path_to_file,
    column_names=["document", "summary"],
    cache_dir='/Documents/Huggin_Face/data'
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Select a sample of the dataset

In [21]:
xsum_sample = xsum_dataset["train"].select(range(1,11))

# Display the sample data

In [22]:
display(xsum_sample.to_pandas())

,document,summary,__index_level_0__
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",00027e965c8264c35cc1bc55556db388da82b07f
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,0002c17436637c4fe1837c935c04de47adb18e9a
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,0003ad6ef0c37534f80b55b4235108024b407f0b
5,He's been accused of making many a fashion fau...,Prime Minister and his family are enjoying an ...,0004306354494f090ee2d7bc5ddbf80b63e80de6
6,By . Daily Mail Reporter . PUBLISHED: . 01:15 ...,NBA star calls for black and Hispanic communit...,0005d61497d21ff37a17751829bd7e3b6e4a7c5c
7,By . Daily Mail Reporter . This is the moment ...,London Midland service had been pulling into T...,0006021f772fad0aa78a977ce4a31b3faa6e6fe5
8,There are a number of job descriptions waiting...,Tony Pulis believes Saido Berahino should look...,00083697263e215e5e7eda753070f08aa374dd45
9,"Canberra, Australia (CNN) -- At first glance, ...",Black box data from Flight 370 could be analyz...,000940f2bb357ac04a236a232156d8b9b18d1667


# Create a summarization pipeline using t5-small

In [23]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    truncation=True,
    model_kwargs={"cache_dir": '/Documents/Huggin_Face/'},
)


# Load the tokenizer for the summarization model

In [24]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Tokenize Input Text and Count Tokens in Example

In [25]:
example_index = 0

input_text = xsum_sample["document"][example_index]

input_tokens = tokenizer.tokenize(input_text)

num_tokens = len(input_tokens)

print("Tokenized Input:", input_tokens)

print("Number of Tokens:", num_tokens)


Tokenized Input: ['▁By', '▁', '.', '▁', 'Associated', '▁Press', '▁', '.', '▁', 'PU', 'BL', 'ISH', 'ED', ':', '▁', '.', '▁14', ':', '11', '▁', 'EST', ',', '▁25', '▁October', '▁2013', '▁', '.', '▁|', '▁', '.', '▁', 'UP', 'DA', 'TED', ':', '▁', '.', '▁15', ':', '36', '▁', 'EST', ',', '▁25', '▁October', '▁2013', '▁', '.', '▁The', '▁bishop', '▁of', '▁the', '▁Far', 'go', '▁Catholic', '▁D', 'iocese', '▁in', '▁North', '▁Dakota', '▁has', '▁exposed', '▁potentially', '▁hundreds', '▁of', '▁church', '▁members', '▁in', '▁Far', 'go', ',', '▁Grand', '▁For', 'k', 's', '▁and', '▁James', 'town', '▁to', '▁the', '▁', 'he', 'pati', 't', 'is', '▁A', '▁virus', '▁in', '▁late', '▁September', '▁and', '▁early', '▁October', '.', '▁The', '▁state', '▁Health', '▁Department', '▁has', '▁issued', '▁an', '▁advisory', '▁of', '▁exposure', '▁for', '▁anyone', '▁who', '▁attended', '▁five', '▁churches', '▁and', '▁took', '▁communion', '.', '▁Bishop', '▁John', '▁Fold', 'a', '▁(', 'pictured', ')', '▁of', '▁the', '▁Far', 'go', '▁C

# Generate a Summary using T5-Small Model with Specified Length Constraints

In [26]:
input_text = xsum_sample["document"][example_index]

generated_summary = summarizer(input_text, max_length=50, min_length=20, do_sample=False)[0]['summary_text']

# Calculate ROUGE score

In [27]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

reference_summary = xsum_sample["summary"][example_index]

scores = scorer.score(generated_summary, reference_summary)

rouge1_scores = scores['rouge1'].fmeasure
rougeL_scores = scores['rougeL'].fmeasure

#Tokenize reference and generated summaries, then calculate METEOR score

In [28]:
import nltk
nltk.download('wordnet')

reference_tokens = [word_tokenize(sent) for sent in sent_tokenize(xsum_sample["summary"][example_index])]
generated_tokens = [word_tokenize(sent) for sent in sent_tokenize(generated_summary)]

reference_tokens_flat = [token for sent in reference_tokens for token in sent]
generated_tokens_flat = [token for sent in generated_tokens for token in sent]

meteor = meteor_score([reference_tokens_flat], generated_tokens_flat)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Print Evaluation Results

Print the generated summary, ROUGE-1 F1 score, ROUGE-L F1 score, and METEOR score.

In [29]:
print("Generated Summary:", generated_summary)

print("ROUGE-1 F1:", rouge1_scores)

print("ROUGE-L F1:", rougeL_scores)

print("METEOR Score:", meteor)

Generated Summary: bishop of the fargo Catholic Diocese in north Dakota has exposed potentially hundreds of church members to the hepatitis A virus . the state health department has issued an advisory of exposure for anyone who attended five churches and took communion 
ROUGE-1 F1: 0.2972972972972973
ROUGE-L F1: 0.18918918918918917
METEOR Score: 0.22598828372854846


# Conclusion

In brief, the provided code represents a sequence-to-sequence (s2s) model with an attention mechanism. It leverages the t5-small model to generate summaries for input documents. Evaluation metrics, including ROUGE and METEOR scores, help us to calculate the quality of the generated summaries against benchmark summaries. This approach is a common technique in natural language processing tasks such as text summarization.

# By: ***Vicky Kumar***